<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Skriptsprachen
### Sommersemester 2021
Prof. Dr. Heiner Giefers

# Generatoren und Iteratoren
Iteratoren erlauben es, alle Elemente eines Datenverbundes zu "durchlaufen", ohne Genaueres über die Implementierung der Datenstruktur zu kennen. Innerhalb einer Programmschleife angewendet, generiert der Iterator eines Objektes eine Sequenz von Referenzen. Diese Referenzen können auf Objekte zeigen, die vom Iterator selbst erzeugt werden, oder auch auf Elemente, die in Datenstrukturen innerhalb des Objektes existieren.

Dieses Übungsblatt behandelt mehrere Aspekte des _Iterierens_ in Python. Zuerst schauen wir uns die sogenannten **Comprehensions** an, ein Konstrukt um aus iterierbaren Objekten Container Objekte zu generieren, die selbst wieder iterierbar sind. **Generatoren** sind Funktionen, die Sequenzen von Objekten nach einem bestimmten Schema erzeugen. Die viel-benutzte _range()_-Funktion ist ein Beispiel für eine Generatorfunktion. Im letzten Teil des Übungsblattes geht es darum, wie man eigene **Iteratoren** innerhalb von Klassen definieren und anwenden kann.

## Comprehensions
Im Übungsblatt zum Thema _Funktionen_ haben Sie lambda-Funktionen kennengelernt. Ein Beispiel für die Anwendung von lambda Funktionen war die eine _map_ Funktion `mymap`:

In [ ]:
def mymap(l, f):
    local_l = []
    for e in l: local_l.append(f(e))
    return local_l

Eine solche Funktion kann benutzt werden, um neue Listen-Objekte aus bestehenden iterierbaren Objekten zu generieren.

In [ ]:
mymap(range(0,10), lambda x: x**2)

Da Operationen dieser Art recht häufig benötigt werden, besitzt Python eingebaute Konstrukte um neue iterierbare Objekte zu erzeugen. **List-Comprehensions** etwa erzeugen neue Listen-Objekte nach folgendem Muster:

In [ ]:
[x**2 for x in range(0,10)]

In [ ]:
satz = ["Mit", "freundlichen", "Grüßen"]
[s[0] for s in satz]

Die eckigen Klammern deuten an, dass das Ergebnis eine List ist. Innerhalb der Klammern steht ein Iterator-Aufruf, dem vorangestellt ist eine _Zuordnungsvorschrift_ wie Sie sie aus lambda-Funktionen kennen. 

Es ist möglich, die _for_-Schleifen der Iteratoren zu verschachteln. Durch einfaches "Anhängen" einer _for_-Schleife, wird eine innere Schleife erzeugt:

In [ ]:
boolean = ['0', '1']
[ "x=%s y=%s z=%s"%(x,y,z) for x in boolean if x!='1' for y in boolean for z in boolean]

Ferner ist es möglich, das Erzeugen eines neuen Elementes an Bedingungen zu knüpfen. Die Bedingungen werden dabei einfach an die entsprechende Schleife angehängt:

In [ ]:
[x*y for x in range(0,10) if x%2==0 for y in range(0,10) if y%2==0 if y>5]

**Aufgabe 1**

**Benutzen Sie List-Comprehensions um eine Wahrheitstabelle für die XOR-Funktion mit 3 Variablen $x$, $y$ und $z$ zu erzeugen**

<pre>
XOR =
['x = 0 y = 0 z = 0 xor = 0',
 'x = 0 y = 0 z = 1 xor = 1',
              .
              .
              .
              .
              .
 'x = 1 y = 1 z = 1 xor = 1']
 </pre>

In [ ]:
XOR = None
# YOUR CODE HERE
raise NotImplementedError()
XOR

In [ ]:
from io import StringIO
from unittest.mock import patch
from IPython import get_ipython
ipython = get_ipython()


with patch('sys.stdout', new_callable=StringIO) as screen:
    ipython.magic('rerun')
    
assert '):' not in screen.getvalue(), 'Use List-Comprehensions, NOT for blocks, to create the XOR list'
tru = [[int(x) for x in y if x in '01'] for y in XOR]
for t in tru: assert ((t[0] != t[1]) != t[2]) == t[3], f'{t[0]} xor {t[1]} xor {t[2]} is not {t[3]}!'

## Generatoren
Comprehensions erzeugen iterierbare Objekte aus bestehenden iterierbaren Objekten. Es gibt aber auch Funktionen, die iterierbare Objekte erzeugen; diese Funktionen nennt man Generatoren.

Die Besonderheit bei Generator-Funktionen ist die `yield` Anweisung. `yield` verhält sich ähnlich zu der `return`-Anweisung, mit dem Unterschied, das `yield` zwar Rückgabewerte erzeugt, die Generator-Funktion aber nicht terminiert. Springt der Kontrollfluss erneut zu der Generator-Funktion, wird sie im vorherigen Zustand fortgesetzt.

In [ ]:
def gruss():
    yield "Mit"
    yield "freundlichen"
    yield "Grüßen"

for wort in gruss(): print(wort, end=' ')

In [ ]:
def zahlen_bis(max):
    i = 1
    while i <= max:
        yield i
        i += 1

for i in zahlen_bis(20):
    print(i, end=' ')

**Aufgabe 2**

**Entwickeln Sie eine Generator-Funktion, die alle ungeraden Zahlen von `nmin` bis `nmax` mit der Schrittweite `nstep` generiert.**

In [ ]:
def ungerade(nmin, nmax, nstep):
    # YOUR CODE HERE
    raise NotImplementedError()

for i in ungerade(4,200,30):
    print(i, end=' ')

In [ ]:
assert '__iter__' in dir(ungerade(1,101,11)), 'The function is not iterable, make sure you use yield!'
for i in ungerade(1,101,11): assert i%2 == 0, 'All items should be even!'
assert len(list(ungerade(1,101,11))) == 5, 'This set has only 5 elements!'

## Iteratoren
Wir haben Iteratoren bereits mehrfach benutzt, bisher aber noch offen gelassen, wie Iteratoren implementiert werden. Instanzen von eingebauten _Container_ Klassen, wie z.B. Listen, Mengen und Dictionaries, können Iteratoren erzeugen. Sie verwenden dazu eine definierte Schnittstelle, die man auch benutzen kann um Objekte von selbst-entwickelten Klassen iterierbar zu machen.
Dazu muss die Klasse die beiden _Magic Methods_ `__iter__()` und `__next__()` definieren. `__iter__()` initialisiert den Iterator und gibt eine Referenz auf das eigene Objekt zurück. Das eigentliche Iterieren durch die Sequenz geschieht durch die Methode `__next__()`. Immer, wenn sie aufgerufen wird, liefert sie das aktuelle Objekt der Sequenz zurück und speichert den neuen Zustand so, dass bei einem folgenden Aufruf das nächste Element der Sequenz berechnet werden kann. Welche Objekte zurückgegeben werden und wie die Reihenfolge der Sequenz strukturiert ist, definiert der Programmierer.
Das Ende eines Iterators wird durch das Erzeugen der `StopIteration`-Ausnahme erreicht. Diese Ausnahme erzeugt keinen Fehler im Interpreter, sondern dient als Abbruchkriterium der for-Schleife, die den Iterator aufruft.

Durch Implementieren der Iterator-Schnittstelle, kann eine Klasse die Funktionalität erhalten, Datensätze innerhalb der Objekte sinnvoll zu durchlaufen, ohne dass der Benutzer die internen Datenstrukturen der Klasse kennen muss. Die Klasse _FifaWm_ im folgenden Beispiel legt in Ihren Objekten ein Tupel-Attribut _sieger_ an, in dem wiederum 2-Tupel der Form (_Jahr_,_Weltmeister_) abgelegt sind. Möchte man eine Liste der Sieger-Länder von einem Objekt der Klasse erhalten, so muss dem Benutzer der Klasse diese Struktur bekannt sein. Durch Implementieren der `__iter__()` und `__next__()` Methoden wird die Klasse iterierbar.

In [ ]:
class FifaWm:
    def __init__(self):
        self.sieger=(
            (1930,"Uruguay"),(1934,"Italien"),(1938,"Italien"),(1950,"Uruguay"),
            (1954,"Deutschland"),(1958,"Brasilien"),(1962,"Brasilien"),(1966,"England"),
            (1970,"Brasilien"),(1974,"Deutschland"),(1978,"Argentinien"),(1982,"Italien"),
            (1986,"Argentinien"),(1990,"Deutschland"),(1994,"Brasilien"),(1998,"Frankreich"),
            (2002,"Brasilien"),(2006,"Italien"),(2010,"Spanien"),(2014,"Deutschland"))
    def __iter__(self):
        #print("__iter__() für die Klasse %s aufgerufen" % self.__class__.__name__)
        self.curr=0
        return self
    def __next__(self):
        if self.curr<len(self.sieger):
            r = self.sieger[self.curr][1]
            self.curr += 1
            return r
        else:
            raise StopIteration

In [ ]:
s = FifaWm()
for land in s: print("%s, " % land, end='')

Man erkennt in der Implementierung der Iterator-Funktionen, dass ein Iterator keine vollständige Kopie des Objektes erzeugt, sondern nur Zeiger "innerhalb" des Objektes selbst verwaltet. Man sollte daher vermeiden, ein und dasselbe Objekt in verschiedenen Schleifen zu iterieren, wie etwa in folgendem Beispiel:

In [ ]:
matches = set()
s = FifaWm()
for land0 in s:
    for land1 in s:
        if land0!=land1:
            matches.add(("%s-%s" % (land0,land1)))
print(matches)

Hier sollten ursprünglich Paarung der Sieger-Länder generiert werden; allerdings sieht man, dass das Ergebnis nicht stimmen kann. Das Problem hier ist, dass die innere Schleife den Iterator koplet "konsumiert". D.h., nachdem die Schleife terminiert, ist in dem Objekt die Abbruchbedingung für den Iterator gültig. Es werden daher keine neuen Objekte erzeugt und so terminiert auch die äußere Schleife nach nur einem Schleifendurchlauf.

Besser ist es an dieser Stelle, 2 Iterator-Ojekte zu erzeugen und diese jeweils in der inneren, bzw. der äußeren Schleife zu verwenden:

In [ ]:
matches = set()
s0 = FifaWm()
s1 = FifaWm()
for land0 in s0:
    for land1 in s1:
        if land0!=land1:
            matches.add(("%s-%s" % (land0,land1)))
print(matches)

**Aufgabe 3**

**Schreiben Sie eine Klasse _Potenzieren_ dessen Konstruktor zwei Parameter besitzt und die entsprechenden Argumente als Attribute `Basis` und `MaxExponent` speichert. Die Klasse soll die Iterator-Schnittstelle in der Form implementieren, dass ein Iterator über die Potenzen der Basis `Basis` von 0 bis `MaxExponent` erzeugt wird**

In [ ]:
class Potenzieren:
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert '__iter__' in dir(Potenzieren(5,10)), 'The function is not iterable, define iter magic method!'
assert '__next__' in dir(Potenzieren(5,10)), 'The function does not have a next magic method!'
assert len(list(Potenzieren(5,10))) == 11, 'This set has only 11 elements!'
for i, p in enumerate(list(Potenzieren(5,10))): assert p == 5**i, f'element number {i} should be {5**i} not {p}'

In [ ]:
p = Potenzieren(7,12)
for p in p: print(p)